In [1]:
# Import needed libraries 

import pandas as pd 
import os
import datetime
from datetime import timedelta
import numpy as np 
from scipy.signal import argrelextrema
import alpaca_trade_api as tradeapi 
import matplotlib.pyplot as plt 
import matplotlib.dates as mpdates
from mplfinance.original_flavor import candlestick_ohlc
from dotenv import load_dotenv
from itertools import islice
from pandas .tseries.offsets import DateOffset

In [2]:
# Load .env file

load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")
ALPACA_ENDPOINT_KEY = os.getenv("ALPACA_END_POINT")


# Create the Alpaca API object
api = tradeapi.REST(
    ALPACA_API_KEY,
    ALPACA_SECRET_KEY,
    api_version="v2",
    base_url= ALPACA_ENDPOINT_KEY
)

In [4]:
# Parameters for Stock Data from Alpacas
# Establish time frame (5 minute)

time_frame = "5min"

# Identify what stock symbol is trading

stock_symbol = "QQQ"

# Identify what start date to begin data analysis

start_date = pd.Timestamp("2021-02-01", tz="America/New_York").isoformat()

# Identify what end date to finalize data analysis

end_date = pd.Timestamp("2021-06-01", tz="America/New_York").isoformat()

In [5]:
print(start_date)
print(end_date)

2021-02-01T00:00:00-05:00
2021-06-01T00:00:00-04:00


In [6]:
# Function to call daily stock data

def get_stock_data(api, stock_symbol, time_frame, start_date, end_date):
    
    # Assuming api.get_bars returns a DataFrame with a 'df' attribute
    
    stock_data = api.get_bars(
        stock_symbol, 
        time_frame, 
        start=start_date, 
        end=end_date
        ).df
    return stock_data

stock_data = get_stock_data(api, stock_symbol, time_frame, start_date, end_date)

# Displays the information pulled for working through code

stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15458 entries, 2021-02-01 09:00:00+00:00 to 2021-05-28 21:55:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        15458 non-null  float64
 1   high         15458 non-null  float64
 2   low          15458 non-null  float64
 3   trade_count  15458 non-null  int64  
 4   open         15458 non-null  float64
 5   volume       15458 non-null  int64  
 6   vwap         15458 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 966.1 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-02-01 09:00:00+00:00,317.78,317.79,317.26,457,317.62,152785,317.615546
2021-02-01 09:05:00+00:00,317.63,317.79,317.54,42,317.75,5968,317.621602
2021-02-01 09:10:00+00:00,317.85,317.90,317.65,41,317.65,5138,317.836724
2021-02-01 09:15:00+00:00,318.15,318.25,317.97,77,318.00,18913,318.110307
2021-02-01 09:20:00+00:00,318.01,318.33,318.00,88,318.11,20816,318.259888


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-05-28 21:25:00+00:00,333.84,333.84,333.80,12,333.80,788,333.814201
2021-05-28 21:30:00+00:00,333.80,333.82,333.80,15,333.82,2008,333.814337
2021-05-28 21:45:00+00:00,333.84,333.84,333.82,11,333.82,1115,333.827005
2021-05-28 21:50:00+00:00,333.83,333.83,333.82,6,333.82,702,333.828604
2021-05-28 21:55:00+00:00,333.82,333.83,333.82,11,333.83,1023,333.823030


In [7]:
# Function to prepare daily stock data to identify double top/bottom patterns and prepare for targets
# Includes establishing a polynomial fit and assigning new columns for localized min/max

# Polynomial Degree

polynomial_degree = 25

def polynomial_min_max_fit(stock_data, polynomial_degree):
    
    min_length = len(stock_data.index)
    x_data = np.arange(min_length)

    # Polynomial fitting
    polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
    polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
    polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
    polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)

    # Evaluate the polynomial fit for plotting
    y_polynomial_open = np.polyval(polynomial_coefficients_open, x_data)
    y_polynomial_high = np.polyval(polynomial_coefficients_high, x_data)
    y_polynomial_low = np.polyval(polynomial_coefficients_low, x_data)
    y_polynomial_close = np.polyval(polynomial_coefficients_close, x_data)

    # Identify local extrema for polynomial fit data (minima and maxima)
    local_poly_minima = argrelextrema(y_polynomial_close, np.less, order=5)[0]
    local_poly_maxima = argrelextrema(y_polynomial_close, np.greater, order=5)[0]

    # Convert the close price polynomial fit data into a dataframe
    # This is done for OHLC poly fit data

    poly_df = pd.DataFrame(y_polynomial_open)
    columns = ['poly_fit_open']
    poly_df.columns = columns
    poly_df = poly_df.assign(poly_fit_high = y_polynomial_high)
    poly_df = poly_df.assign(poly_fit_low = y_polynomial_low)
    poly_df = poly_df.assign(poly_fit_close = y_polynomial_close)

    # Reset the index of the original updated ticker dataframe to concat with the polynomial dataframe that does not include a timeseries
    # This will ensure that the indexed intergers of the ploynomial fit align with the time each data point corresponds to

    updated_stock_data = stock_data.reset_index()
    updated_stock_data = pd.concat([updated_stock_data, poly_df], axis='columns', join='inner')
    updated_stock_data.head()

    # Add minima and maxima column to the DataFrame

    updated_stock_data["minima"] = 0
    updated_stock_data["maxima"] = 0
    updated_stock_data.head()

    # Mark rows with local minima as 1 in the 'minima' column

    for index in local_poly_minima:
        updated_stock_data.at[index, 'minima'] = -1

    for index in local_poly_maxima:
        updated_stock_data.at[index, "maxima"] = 1

    # Create Target Columns - Double Top Target & Double Bottom Target
        
    updated_stock_data["dbl_top_target"] = 0
    updated_stock_data["dbl_bot_target"] = 0

    # # Plot data: COMMENTED OUT TO NOT REPEAT PLOTS AT END OF PROGRAM
    # # Plot the stock data and identified minima

    # plt.figure(figsize=(15, 5))
    # plt.plot(stock_data.index, stock_data["close"], label='Close Prices', alpha=0.7)

    # # Plot polynomial fit

    # plt.plot(stock_data.index[:min_length], y_polynomial_close, '-', markersize=1.0, color='black', alpha=0.9, label='Polynomial Fit')

    # # Plot red dots at local minima and blue dots at local maxima

    # plt.scatter(stock_data.index[local_poly_minima], y_polynomial_close[local_poly_minima], color='red', label='Local Minima')
    # plt.scatter(stock_data.index[local_poly_maxima],y_polynomial_close[local_poly_maxima], color="blue", label = "Local Maxima")
    
    return updated_stock_data

# For data checking/confirmation
updated_stock_data = polynomial_min_max_fit(stock_data,polynomial_degree)
updated_stock_data.head()

C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_16348\2367383943.py:14: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_16348\2367383943.py:15: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_16348\2367383943.py:16: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_16348\2367383943.py:17: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)


,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
0,2021-02-01 09:00:00+00:00,317.78,317.79,317.26,457,317.62,152785,317.615546,312.930406,313.157730,312.744286,312.916258,0,0,0,0
1,2021-02-01 09:05:00+00:00,317.63,317.79,317.54,42,317.75,5968,317.621602,313.056023,313.282869,312.869895,313.042180,0,0,0,0
2,2021-02-01 09:10:00+00:00,317.85,317.90,317.65,41,317.65,5138,317.836724,313.180801,313.407171,312.994663,313.167260,0,0,0,0
3,2021-02-01 09:15:00+00:00,318.15,318.25,317.97,77,318.00,18913,318.110307,313.304743,313.530640,313.118597,313.291503,0,0,0,0
4,2021-02-01 09:20:00+00:00,318.01,318.33,318.00,88,318.11,20816,318.259888,313.427853,313.653279,313.241700,313.414912,0,0,0,0


In [ ]:
# # Shift dataframe by one datapoint to set target columns to train  ML model 

# updated_stock_data['close_lagged'] = updated_stock_data['close'].shift(1)
# updated_stock_data['high_lagged'] = updated_stock_data['high'].shift(1)
# updated_stock_data['low_lagged'] = updated_stock_data['low'].shift(1)
# updated_stock_data['open_lagged'] = updated_stock_data['open'].shift(1)
# updated_stock_data['trade_count_lagged'] = updated_stock_data['trade_count'].shift(1)
# updated_stock_data['volume_lagged'] = updated_stock_data['volume'].shift(1)
# updated_stock_data['vwap_lagged'] = updated_stock_data['vwap'].shift(1)
# updated_stock_data['poly_open_lagged'] = updated_stock_data['poly_fit_open'].shift(1)
# updated_stock_data['poly_high_lagged'] = updated_stock_data['poly_fit_high'].shift(1)
# updated_stock_data['poly_low_lagged'] = updated_stock_data['poly_fit_low'].shift(1)
# updated_stock_data['poly_close_lagged'] = updated_stock_data['poly_fit_close'].shift(1)
# updated_stock_data['minima_lagged'] = updated_stock_data['minima'].shift(1)
# updated_stock_data['maxima_lagged'] = updated_stock_data['maxima'].shift(1)

# display(updated_stock_data.head())
# display(updated_stock_data.tail())

In [ ]:
# # Plot data
# # Plot the stock data and identified minima

# plt.figure(figsize=(15, 5))
# plt.plot(stock_data.index, stock_data["close"], label='Close Prices', alpha=0.7)

# # # Plot polynomial fit

# plt.plot(stock_data.index[:min_length], y_polynomial_close, '-', markersize=1.0, color='black', alpha=0.9, label='Polynomial Fit')

# # # Plot red dots at local minima and blue dots at local maxima

# plt.scatter(stock_data.index[local_poly_minima], y_polynomial_close[local_poly_minima], color='red', label='Local Minima')
# plt.scatter(stock_data.index[local_poly_maxima],y_polynomial_close[local_poly_maxima], color="blue", label = "Local Maxima")

In [8]:
# Define the time independent DF for double top/bottom identification
# Timestamp remains a column, but can identify pattern from peak to peak immediately

def time_independent_data(updated_stock_data):

    # Define and establish time independent DF

    time_independent_df = []
    time_independent_df = pd.DataFrame(time_independent_df, columns = ["timestamp","close","high","low","trade_count","open","volume","vwap","poly_fit_open","poly_fit_high","poly_fit_low","poly_fit_close","minima","maxima", "dbl_top_target","dbl_bot_target"])

    # Loop to iterate through all rows of stock data and write min/max to new DF

    for index, row in islice(updated_stock_data.iterrows(), 0, None):

        # Assign to rows only those that contain local min or max

        if (updated_stock_data.at[index,"minima"] == -1):
            time_independent_df.loc[index] = row
            a = updated_stock_data.iloc[index]["poly_fit_close"]
        elif (updated_stock_data.at[index,"maxima"] == 1):
            time_independent_df.loc[index] = row
            a = updated_stock_data.iloc[index]["poly_fit_close"]

    # Reset time dependent index
    # Set new time independent index

    time_independent_df.reset_index(inplace = True)
    time_independent_df.rename(columns={"index":"time_dependent_index"}, inplace = True)

    return time_independent_df

# Data confirmation/check

time_independent_df = time_independent_data(updated_stock_data)
time_independent_df.head(10)

,time_dependent_index,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
0,1536,2021-02-11 11:50:00+00:00,334.1,334.22,334.06,51,334.16,5938,334.109267,335.489916,335.593558,335.376124,335.490674,0,1,0,0
1,4424,2021-03-05 19:40:00+00:00,307.535,308.24,307.14,9474,307.99,897074,307.727024,309.331801,309.647273,309.01703,309.327414,-1,0,0,0
2,5954,2021-03-17 22:20:00+00:00,322.15,322.22,322.15,40,322.22,8299,322.186672,318.284073,318.479015,318.075503,318.280644,0,1,0,0
3,7088,2021-03-25 23:00:00+00:00,311.59,311.64,311.54,18,311.58,3860,311.613834,313.906709,314.095617,313.700954,313.903994,-1,0,0,0
4,9364,2021-04-14 10:05:00+00:00,341.19,341.26,341.18,30,341.23,2466,341.214217,339.617838,339.733633,339.493057,339.617857,0,1,0,0
5,10262,2021-04-20 23:20:00+00:00,335.1,335.13,335.09,16,335.11,2150,335.111153,337.793903,337.933293,337.649807,337.791455,-1,0,0,0
6,11234,2021-04-28 11:05:00+00:00,340.03,340.06,339.86,93,339.96,20839,339.952775,339.996713,340.109649,339.872701,339.999674,0,1,0,0
7,13705,2021-05-17 16:35:00+00:00,323.24,323.27,322.7099,1683,322.73,169100,323.045953,321.629896,321.825519,321.441034,321.629637,-1,0,0,0
8,14951,2021-05-26 11:05:00+00:00,334.12,334.24,334.04,62,334.24,9224,334.163679,333.620895,333.724459,333.513186,333.61944,0,1,0,0
9,15103,2021-05-26 23:45:00+00:00,333.98,333.99,333.93,41,333.95,10923,333.962246,333.520122,333.622286,333.41438,333.521461,-1,0,0,0


In [9]:
# Function to identify daily double top/bottom patterns
# Inherently there should only be 1 identification of the pattern as it is considered a trend reversal pattern
# The loop will break once the pattern is identified

def identify_double_patterns(time_independent_df, updated_stock_data):

    # Initialize variables to identify double top/bottom patterns
    # Time dependent variable x_0 will always begin at a local min/max which also coincides
    # With the start of a trend into a potential double top/bottom
    
    x_0 = 0

    # Initiation of double top/bottom variable
    
    a = 0

    # First peak/valley of double top/bottom pattern
    
    b = 0

    # Trough/peak of double top/bottom pattern

    c = 0

    # Second peak/vallye of double top/bottom pattern
    
    d = 0

    # Trigger of double top/bottom pattern
    
    e = 0

    # Final time increment to finalize and trigger double top signal
    
    x_f = 0

    # Read through code to identify double top/bottom and assign to target columns.

    for index, row in islice(time_independent_df.iterrows(), 0, len(time_independent_df) - 4):

        # Check for double top
        # If found then assigns x_f final time for writing to targets in time dependent dataframe

        if (time_independent_df.at[index,"minima"] == -1):
            a = time_independent_df.iloc[index]["poly_fit_close"]
            b = time_independent_df.iloc[index + 1]["poly_fit_close"]
            c = time_independent_df.iloc[index + 2]["poly_fit_close"]
            d = time_independent_df.iloc[index + 3]["poly_fit_close"]
            e = time_independent_df.iloc[index + 4]["poly_fit_close"]
            x_0 = time_independent_df.iloc[index]["time_dependent_index"]

            if (time_independent_df.iloc[index + 1]["poly_fit_low"])*.95 < d < (time_independent_df.iloc[index + 1]["poly_fit_high"])*1.05 and (e <= c):
                x_f = time_independent_df.iloc[index + 4]["time_dependent_index"]
                updated_stock_data.at[x_f, "long_short_target"] = -1
                while (x_0 < x_f + 1):
                    updated_stock_data.at[x_0, "dbl_top_target"] = 1
                    x_0 = x_0 + 1
                break
            
            

        # Check for double bottom
        # If found then assigns x_f final time for writing to targets in time dependent dataframe
            
        elif (time_independent_df.at[index,"maxima"] == 1):
            a = time_independent_df.iloc[index]["poly_fit_close"]
            b = time_independent_df.iloc[index + 1]["poly_fit_close"]
            c = time_independent_df.iloc[index + 2]["poly_fit_close"]
            d = time_independent_df.iloc[index + 3]["poly_fit_close"]
            e = time_independent_df.iloc[index + 4]["poly_fit_close"]
            x_0 = time_independent_df.iloc[index]["time_dependent_index"] 

            if (time_independent_df.iloc[index+1]["poly_fit_low"])*.95 < d < (time_independent_df.iloc[index+1]["poly_fit_high"])*1.05 and (e >= c):
                x_f_min = time_independent_df.iloc[index + 4]["time_dependent_index"]
                updated_stock_data.at[x_f, "long_short_target"] = 1
                while (x_0 < x_f + 1):
                    updated_stock_data.at[x_0, "dbl_bot_target"] = 1
                    x_0 = x_0 + 1
                break
            
    return updated_stock_data, x_f

updated_stock_data, x_f = identify_double_patterns(time_independent_df,updated_stock_data)

print(x_f)

0


In [10]:
print(updated_stock_data.columns)

Index(['timestamp', 'close', 'high', 'low', 'trade_count', 'open', 'volume',
       'vwap', 'poly_fit_open', 'poly_fit_high', 'poly_fit_low',
       'poly_fit_close', 'minima', 'maxima', 'dbl_top_target',
       'dbl_bot_target', 'long_short_target'],
      dtype='object')


In [12]:
# Loop to collect and assess daily data one day at a time

master_df = {
    'timestamp': [],  # List of timestamps
    'close': [],      # List of close prices
    'high': [],       # List of high prices
    'low': [],        # List of low prices
    'trade_count': [], # List of trade counts
    'open': [],       # List of open prices
    'volume': [],     # List of volumes
    'vwap': [],       # List of volume-weighted average prices
    'poly_fit_open': [],    # List of polynomial fit open prices
    'poly_fit_high': [],    # List of polynomial fit high prices
    'poly_fit_low': [],     # List of polynomial fit low prices
    'poly_fit_close': [],   # List of polynomial fit close prices
    'minima': [],           # List of minima
    'maxima': [],           # List of maxima
    'dbl_top_target': [],   # List of double top targets
    'dbl_bot_target': [],    # List of double bottom targets
    'long_short_target': []       # List of ML Model target
}

daily_dataframes = []

master_df = pd.DataFrame(master_df)

current_date = pd.to_datetime(start_date)
next_day_date = current_date + pd.offsets.BDay(1)

print(current_date)
print(next_day_date)

while current_date <= (pd.to_datetime(end_date)):

    # next_day_date = next_day_date + pd.offsets.BDay(1)
    # current_date = current_date + pd.offsets.BDay(1)
    # Convert current_date and next_day_date to ISO format for API call

    current_date_iso = current_date.isoformat()
    # next_day_date = current_date + pd.offsets.BDay(1)
    next_day_date_iso = next_day_date.isoformat()

    #current_date = datetime.datetime.fromisoformat(current_date_iso) + pd.offsets.BDay(1)
    
    # Fetch stock data for the current day

    stock_data = get_stock_data(api, stock_symbol, time_frame, current_date_iso, next_day_date_iso)

    if len(stock_data.index) == 0:
        next_day_date = next_day_date + pd.offsets.BDay(1)
        current_date = current_date + pd.offsets.BDay(1)
        continue
    
    # Run polynomial fit function for updated stock data

    updated_stock_data = polynomial_min_max_fit(stock_data, polynomial_degree)
    
    # Isolate Mins/maxes

    time_independent_df = time_independent_data(updated_stock_data)

    # Updated stock data with double tops/bots identified by function
    # x_f is the double top/bot trigger row(index) for current day's double top/bot

    updated_stock_data, x_f = identify_double_patterns(time_independent_df, updated_stock_data)
    
    # Perform your analysis or call your functions here
    # e.g., identify_double_patterns(time_independent_df, updated_stock_data)

    daily_dataframes.append(updated_stock_data)

    master_df = pd.concat(daily_dataframes, ignore_index=True)
    # display(master_df.tail())
    # Increment to the next day

    next_day_date = next_day_date + pd.offsets.BDay(1)
    current_date = current_date + pd.offsets.BDay(1)
    # next_day_date = pd.Timestamp(next_day_date_iso) + pd.offsets.BDay(1)
    
    # print(current_date)
    # print(next_day_date)
    

2021-02-01 00:00:00-05:00
2021-02-02 00:00:00-05:00


C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_16348\2367383943.py:14: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_16348\2367383943.py:15: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_16348\2367383943.py:16: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_16348\2367383943.py:17: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_16348\2367383943.py:14: RankWarning: Pol

In [13]:
# Shift master dataframe by one datapoint to set target columns to train  ML model 

master_df['close_lagged'] = master_df['close'].shift(1)
master_df['high_lagged'] = master_df['high'].shift(1)
master_df['low_lagged'] = master_df['low'].shift(1)
master_df['open_lagged'] = master_df['open'].shift(1)
master_df['trade_count_lagged'] = master_df['trade_count'].shift(1)
master_df['volume_lagged'] = master_df['volume'].shift(1)
master_df['vwap_lagged'] = master_df['vwap'].shift(1)
master_df['poly_open_lagged'] = master_df['poly_fit_open'].shift(1)
master_df['poly_high_lagged'] = master_df['poly_fit_high'].shift(1)
master_df['poly_low_lagged'] = master_df['poly_fit_low'].shift(1)
master_df['poly_close_lagged'] = master_df['poly_fit_close'].shift(1)
master_df['minima_lagged'] = master_df['minima'].shift(1)
master_df['maxima_lagged'] = master_df['maxima'].shift(1)
master_df['dbl_top_target_lagged'] = master_df['dbl_top_target'].shift(1)
master_df['dbl_bot_target_lagged'] = master_df['dbl_bot_target'].shift(1)
master_df['long_short_target_lagged'] = master_df['long_short_target'].shift(1)

In [14]:
master_df.info()
print(len(master_df))
print(type(master_df))
print(sum(master_df["dbl_top_target"]))
print(sum(master_df["dbl_bot_target"]))
print(sum(master_df["long_short_target"]))
display(master_df.head(50))
display(master_df.tail(50))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15458 entries, 0 to 15457
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   timestamp                 15458 non-null  datetime64[ns, UTC]
 1   close                     15458 non-null  float64            
 2   high                      15458 non-null  float64            
 3   low                       15458 non-null  float64            
 4   trade_count               15458 non-null  int64              
 5   open                      15458 non-null  float64            
 6   volume                    15458 non-null  int64              
 7   vwap                      15458 non-null  float64            
 8   poly_fit_open             15458 non-null  float64            
 9   poly_fit_high             15458 non-null  float64            
 10  poly_fit_low              15458 non-null  float64            
 11  poly_fit_close 

,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,...,vwap_lagged,poly_open_lagged,poly_high_lagged,poly_low_lagged,poly_close_lagged,minima_lagged,maxima_lagged,dbl_top_target_lagged,dbl_bot_target_lagged,long_short_target_lagged
0,2021-02-01 09:00:00+00:00,317.78,317.7900,317.260,457,317.6200,152785,317.615546,317.589178,317.713873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-02-01 09:05:00+00:00,317.63,317.7900,317.540,42,317.7500,5968,317.621602,317.741114,317.920427,...,317.615546,317.589178,317.713873,317.194772,317.668435,0.0,0.0,0.0,0.0,NaN
2,2021-02-01 09:10:00+00:00,317.85,317.9000,317.650,41,317.6500,5138,317.836724,317.855104,318.045420,...,317.621602,317.741114,317.920427,317.621386,317.859682,0.0,0.0,0.0,0.0,NaN
3,2021-02-01 09:15:00+00:00,318.15,318.2500,317.970,77,318.0000,18913,318.110307,317.933155,318.110569,...,317.836724,317.855104,318.045420,317.816195,317.940566,0.0,0.0,0.0,0.0,NaN
4,2021-02-01 09:20:00+00:00,318.01,318.3300,318.000,88,318.1100,20816,318.259888,317.980292,318.134466,...,318.110307,317.933155,318.110569,317.885216,317.961841,0.0,1.0,0.0,0.0,NaN
5,2021-02-01 09:25:00+00:00,318.11,318.1600,318.000,42,318.0800,7704,318.125444,318.002786,318.132133,...,318.259888,317.980292,318.134466,317.893533,317.954719,0.0,0.0,0.0,0.0,NaN
6,2021-02-01 09:30:00+00:00,318.06,318.1700,318.060,157,318.0900,32477,318.148594,318.007017,318.115040,...,318.125444,318.002786,318.132133,317.878399,317.937052,0.0,0.0,0.0,0.0,NaN
7,2021-02-01 09:35:00+00:00,317.84,318.0800,317.810,32,318.0500,5522,317.879103,317.998796,318.091419,...,318.148594,318.007017,318.115040,317.858895,317.917933,0.0,0.0,0.0,0.0,NaN
8,2021-02-01 09:40:00+00:00,317.71,317.9100,317.700,61,317.8600,14239,317.791600,317.983014,318.066738,...,317.879103,317.998796,318.091419,317.842892,317.901042,0.0,0.0,0.0,0.0,NaN
9,2021-02-01 09:45:00+00:00,317.49,317.8100,317.490,25,317.8100,3617,317.720274,317.963520,318.044240,...,317.791600,317.983014,318.066738,317.831919,317.887018,0.0,0.0,0.0,0.0,NaN


,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,...,vwap_lagged,poly_open_lagged,poly_high_lagged,poly_low_lagged,poly_close_lagged,minima_lagged,maxima_lagged,dbl_top_target_lagged,dbl_bot_target_lagged,long_short_target_lagged
15408,2021-05-28 17:35:00+00:00,334.5300,334.8700,334.4800,1967,334.8050,221634,334.701173,334.660179,334.726937,...,334.758369,334.746176,334.804980,334.574491,334.666637,0.0,0.0,0.0,0.0,NaN
15409,2021-05-28 17:40:00+00:00,334.4800,334.5650,334.3800,1885,334.5200,267678,334.483766,334.584460,334.659497,...,334.701173,334.660179,334.726937,334.493680,334.591795,0.0,0.0,0.0,0.0,NaN
15410,2021-05-28 17:45:00+00:00,334.3000,334.5000,334.1800,2514,334.4800,347212,334.296986,334.521328,334.604430,...,334.483766,334.584460,334.659497,334.424441,334.529085,0.0,0.0,0.0,0.0,NaN
15411,2021-05-28 17:50:00+00:00,334.4600,334.4600,334.2500,1445,334.3050,155678,334.376539,334.472540,334.562954,...,334.296986,334.521328,334.604430,334.368759,334.480261,0.0,0.0,0.0,0.0,NaN
15412,2021-05-28 17:55:00+00:00,334.4300,334.4900,334.3200,1317,334.4500,128026,334.409440,334.439208,334.535686,...,334.376539,334.472540,334.562954,334.328047,334.446471,0.0,0.0,0.0,0.0,NaN
15413,2021-05-28 18:00:00+00:00,334.4799,334.4806,334.3400,1920,334.4300,214047,334.435129,334.421748,334.522618,...,334.409440,334.439208,334.535686,334.303073,334.428198,0.0,0.0,0.0,0.0,NaN
15414,2021-05-28 18:05:00+00:00,334.3900,334.5400,334.3300,1981,334.4700,250928,334.444026,334.419846,334.523132,...,334.435129,334.421748,334.522618,334.293920,334.425223,-1.0,0.0,0.0,0.0,NaN
15415,2021-05-28 18:10:00+00:00,334.3399,334.4300,334.2100,2164,334.3900,299300,334.342129,334.432470,334.536026,...,334.444026,334.419846,334.523132,334.299962,334.436628,0.0,0.0,0.0,0.0,NaN
15416,2021-05-28 18:15:00+00:00,334.4130,334.4700,334.2700,1787,334.3200,201644,334.396204,334.457903,334.559582,...,334.342129,334.432470,334.536026,334.319887,334.460823,0.0,0.0,0.0,0.0,NaN
15417,2021-05-28 18:20:00+00:00,334.4799,334.4900,334.3200,1839,334.4000,224579,334.404196,334.493825,334.591645,...,334.396204,334.457903,334.559582,334.351738,334.495607,0.0,0.0,0.0,0.0,NaN
